In [131]:
import osmnx as ox
import networkx as nx

import geopandas as gpd
import pandas as pd
import numpy as np
from statistics import mean

import matplotlib.pyplot as plt
import shapely
import folium

import os

In [133]:
location = 'Richmond, California, USA'
richmond_center = [37.943882, -122.35342]

ca_teale_albers_nad83 = 'NAD_1983_California_Teale_Albers_FtUS'
nad83 = 'EPSG:4629'

wgs84 = 'EPSG:4326'
wgs84_m = 'EPSG:4798'

In [135]:
richmond_basemap = folium.Map(location = richmond_center, zoom_start = 12, width = '50%', height = '50%', tiles = 'OpenStreetMap')

#richmond_basemap

In [136]:
def plot_gdf_with_background(gdf, zoom=12, center = None):
    
    if center is not None:
        map_center = center
    else:
        map_center_lon = gdf['geometry'].x.mean()
        map_center_lat = gdf['geometry'].y.mean()
        map_center = [map_center_lat, map_center_lon]
    
    map_with_background = folium.Map(map_center, zoom_start = zoom, width = '50%', height = '50%', 
                                     tiles = 'OpenStreetMap')
    
    folium.GeoJson(gdf).add_to(map_with_background)
    
    return(map_with_background)

## Read in and Clean Necessary Data

In [137]:
import warnings

warnings.filterwarnings('ignore')

#graph_raw = ox.graph_from_place(location, network_type = 'drive')

graph = ox.project_graph(graph_raw, to_crs = wgs84)
graph = ox.add_edge_speeds(graph)
graph = ox.add_edge_travel_times(graph)

graph_nodes_gdf, graph_edges_gdf = ox.graph_to_gdfs(graph, nodes = True, edges = True)

#ox.plot_graph(graph)

In [139]:
hubs_path = os.path.join(os.getcwd(), 'data', 'candidate_site_campuses_2021-11-17', 'candidate_site_campuses.csv')

hubs_df_raw = pd.read_csv(hubs_path)
hubs_df_raw = hubs_df_raw.loc[hubs_df_raw['cat_site'] != 'X', ['id_site', 'SQFT_ROOF', 'LON', 'LAT']]

hubs_geom = gpd.points_from_xy(hubs_df_raw.LON, hubs_df_raw.LAT, crs = wgs84)
hubs_gdf = gpd.GeoDataFrame(hubs_df_raw, geometry = hubs_geom, crs = wgs84)

#plot_gdf_with_background(hubs_gdf, 13, richmond_center)

In [141]:
cengeos_path = os.path.join(os.getcwd(), 'data', 'bg_ca_19', 'blockgroup_CA_19.shp')

cengeos_gdf = gpd.read_file(cengeos_path)
cengeos_gdf = cengeos_gdf.to_crs(hubs_gdf.crs)

#cengeos_gdf.plot()

In [149]:
cengeos_lons = [float(intpt) for intpt in cengeos_gdf['INTPTLON']]
cengeos_lats = [float(intpt) for intpt in cengeos_gdf['INTPTLAT']]

cengeos = pd.DataFrame(cengeos_gdf[['GISJOIN', 'COUNTYFP']])
cengeos['LON'] = cengeos_lons
cengeos['LAT'] = cengeos_lats

cengeos_pt_geom = gpd.points_from_xy(x = cengeos.LON,y = cengeos.LAT, crs = wgs84)
cengeos_pt_gdf = gpd.GeoDataFrame(cengeos, geometry = cengeos_pt_geom, crs = wgs84)

#plot_gdf_with_background(cengeos_pt_gdf, 13, richmond_center)

## Restrict to Hubs within Desired Area

In [144]:
lon_max = graph_nodes_gdf['lon'].max()
lon_min = graph_nodes_gdf['lon'].min()
lon_avg = (lon_max + lon_min)/2

lat_max = graph_nodes_gdf['lat'].max()
lat_min = graph_nodes_gdf['lat'].min()
lat_avg = (lat_min + lat_max)/2

bbox_coords = [(lon_min, lat_min), (lon_min, lat_max), (lon_max, lat_max), (lon_max, lat_min)]
bbox_poly = shapely.geometry.Polygon(bbox_coords)

In [146]:
hubs_gdf_bbox = hubs_gdf[hubs_gdf.within(bbox_poly)]

#plot_gdf_with_background(hubs_gdf_bbox)

In [189]:
cengeos_pt_gdf_bbox = cengeos_pt_gdf[cengeos_pt_gdf.within(bbox_poly)]

### REMOVE THE FOLLOWING LINE AFTER TESTING!!! - selecting random BGs for matrix
cengeos_pt_gdf_bbox = cengeos_pt_gdf_bbox.iloc[[0, 3, 12, 20, 48, 65], ]

#plot_gdf_with_background(cengeos_pt_gdf_bbox, center = richmond_center)

## Build Distance Matrix for Desired Area

In [190]:
matrix_dist_to_hubs = np.NaN*np.zeros((len(cengeos_gdf_bbox), len(hubs_gdf_bbox))) #what does np.zeros do?

In [191]:
cengeos_buffer = cengeos_pt_gdf_bbox.buffer(0.025)

In [192]:
n_cengeos = len(cengeos_pt_gdf_bbox)

In [204]:
for cengeo in range(0, n_cengeos):
    
    cengeo_buffer = cengeos_buffer.iloc[cengeo]
    
    hubs_nearby_gdf = hubs_gdf_bbox[hubs_gdf_bbox.within(cengeo_buffer)]
    
    n_hubs_nearby = len(hubs_nearby_gdf)

In [210]:
for hub in range(0, n_hubs_nearby):
    
    hub_geom = hubs_nearby_gdf.iloc[hub]['geometry']
    
    print(hub_geom)

POINT (-122.34621155 37.95213657)
POINT (-122.3416791 37.95243215)
POINT (-122.34535805 37.94236182)
POINT (-122.36414091 37.93221329)
POINT (-122.34239007 37.93425788)
POINT (-122.35521553 37.9260103)
POINT (-122.35518118 37.92549379)
POINT (-122.31556282 37.92980015)
POINT (-122.33723978 37.93343146)
POINT (-122.34233415 37.92226036)
POINT (-122.33083995 37.93279054)
POINT (-122.31894519 37.91933511)
POINT (-122.3440602 37.93589453)
POINT (-122.32269526 37.91997057)
POINT (-122.32883909 37.92487027)
POINT (-122.3590218 37.92753124)
POINT (-122.33157023 37.92919212)
POINT (-122.32668233 37.91847736)
POINT (-122.32382748 37.9151525)
POINT (-122.3602583 37.93379469)
POINT (-122.33708026 37.92244216)
POINT (-122.33266617 37.94959289)
POINT (-122.34514871 37.93843666)
POINT (-122.34605073 37.93873566)
POINT (-122.3357235 37.93435665)
POINT (-122.33212583 37.93327848)
POINT (-122.32506167 37.93276733)
POINT (-122.35767132 37.92766907)
POINT (-122.35722098 37.93054564)
